<a href="https://colab.research.google.com/github/ssempax1/DataMining/blob/master/Project_1_Data_Mining_Summer_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1 Summary

kjkkj

*   List item
*   List item



# Question 2

## Installing Package

In [1]:
!pip install aif360

     |████████████████████████████████| 174kB 4.8MB/s 


In [0]:
#code to load data file

In [7]:
!wget -O ../usr/local/lib/python3.6/dist-packages/aif360/data/raw/german/german.data https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data

--2020-06-12 07:41:20--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79793 (78K) [application/x-httpd-php]
Saving to: ‘../usr/local/lib/python3.6/dist-packages/aif360/data/raw/german/german.data’

../usr/local/lib/py 100%[===================>]  77.92K  --.-KB/s    in 0.06s   

2020-06-12 07:41:20 (1.26 MB/s) - ‘../usr/local/lib/python3.6/dist-packages/aif360/data/raw/german/german.data’ saved [79793/79793]



## Set 1: Importing Statements

In [0]:
import sys
sys.path.insert(1, "../")  
import numpy as np
np.random.seed(0)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_german
from aif360.algorithms.preprocessing import DisparateImpactRemover
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
# Explainers
from aif360.explainers import MetricTextExplainer
#from common_utils import compute_metrics

## Step 2: Set bias detection options, loading dataset, and splitting train and test dataset

In [0]:
# This dataset also contains protected attribute for "sex" which we do not consider in this evaluation
# age >=25 is considered privileged
# ignore sex-related attributes
dataset_orig = GermanDataset(protected_attribute_names=['age'],privileged_classes=[lambda x: x >= 25],features_to_drop=['personal_status', 'sex'] )   
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]


## Step 3 Computing fairness metric on original training dataset

In [12]:
## Step 3 Computing fairness metric on original training dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train,unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.169905


## Step 4 Mitigating bias by transforming the original dataset

In [0]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

## Step 5 Computing fairness metric on transformed dataset

In [15]:
## Step 5 Computing fairness metric on transformed dataset
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train,unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


# Question 3

### Analyzing train dataset without AIF360 package

In [16]:
def to_dataframes(aif360_ds):
  X = pd.DataFrame(aif360_ds.features, columns=aif360_ds.feature_names)
  y = pd.DataFrame(aif360_ds.labels.ravel(), columns=aif360_ds.label_names)
  Xy = pd.concat([X,y],axis=1)
  return Xy

train_XY = to_dataframes(dataset_orig_train)
test_XY= to_dataframes(dataset_orig_test)

privil = len(train_XY.loc[train_XY['age']==1])
unprivil = len(train_XY.loc[train_XY['age']==0])

privil_fav = len(train_XY.loc[(train_XY['age']==1) & (train_XY['credit']==1)])
unprivil_fav = len(train_XY.loc[(train_XY['age']==0) & (train_XY['credit']==1)])

print("privileged: {'age': 1}",privil," unprivileged: {'age': 0}",unprivil)
print("privileged_favorable:",privil_fav," unprivileged_favorable:",unprivil_fav)
ratio_unpriv_fav = unprivil_fav / unprivil
ratio_priv_fav = privil_fav / privil
ratio_disparate_impact=(ratio_unpriv_fav/ratio_priv_fav)*100
print("Ratio Unprivileged Favorable:",ratio_unpriv_fav)
print("Ratio Privileged Favorable:",ratio_priv_fav)
print("Disparate Impact:",ratio_disparate_impact)
#plt.hist(dataset_orig_train.labels)
#X_original


privileged: {'age': 1} 587  unprivileged: {'age': 0} 113
privileged_favorable: 427  unprivileged_favorable: 63
Ratio Unprivileged Favorable: 0.5575221238938053
Ratio Privileged Favorable: 0.727427597955707
Disparate Impact: 76.64297113013201


## Using Disparate Impact and Disparate Impact Remove

---



In [17]:
## Step 3 Installing BlackBoxAuditing for use Disparate Impact Remove 
!pip3 install BlackBoxAuditing

     |████████████████████████████████| 2.6MB 4.8MB/s 
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394767 sha256=579693de8b8f5469dec666c9169d1e5434c9ddc9beed1896db55a05e351d6d11
  Stored in directory: /root/.cache/pip/wheels/66/82/7b/ac2a79b8caf97e15ed415162a7f272cbba1e2e2c851fa76ae3
Successfully built BlackBoxAuditing


In [18]:
## Step 3 Computing Fairness Metric
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train,unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Disparate Impact unprivileged and privileged groups = %f" % metric_orig_train.disparate_impact())

#### Original training dataset

Disparate Impact unprivileged and privileged groups = 0.766430


In [0]:
## Step 4 Mitigation bias using Disparate Impact Remove
DI = DisparateImpactRemover(repair_level=1.0)
dataset_transf_train = DI.fit_transform(dataset_orig_train)

In [20]:
## Step 5 Computing Fairness metric on transformed dataset
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train,unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.disparate_impact())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.766430


# Revising Original Dataset

In [0]:
# Reading original dataset for a better understanding
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
names = ['existingchecking', 'duration', 'credithistory', 'purpose', 'creditamount', 
         'savings', 'employmentsince', 'installmentrate', 'statussex', 'otherdebtors', 
         'residencesince', 'property', 'age', 'otherinstallmentplans', 'housing', 
         'existingcredits', 'job', 'peopleliable', 'telephone', 'foreignworker', 'classification']
data = pd.read_csv(url,names = names, delimiter=' ')

In [22]:
data.head(10)

,existingchecking,duration,credithistory,purpose,creditamount,savings,employmentsince,installmentrate,statussex,otherdebtors,residencesince,property,age,otherinstallmentplans,housing,existingcredits,job,peopleliable,telephone,foreignworker,classification
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,2
5,A14,36,A32,A46,9055,A65,A73,2,A93,A101,4,A124,35,A143,A153,1,A172,2,A192,A201,1
6,A14,24,A32,A42,2835,A63,A75,3,A93,A101,4,A122,53,A143,A152,1,A173,1,A191,A201,1
7,A12,36,A32,A41,6948,A61,A73,2,A93,A101,2,A123,35,A143,A151,1,A174,1,A192,A201,1
8,A14,12,A32,A43,3059,A64,A74,2,A91,A101,4,A121,61,A143,A152,1,A172,1,A191,A201,1
9,A12,30,A34,A40,5234,A61,A71,4,A94,A101,2,A123,28,A143,A152,2,A174,1,A191,A201,2


In [23]:
print(data.shape)
print (data.columns)

(1000, 21)
Index(['existingchecking', 'duration', 'credithistory', 'purpose',
       'creditamount', 'savings', 'employmentsince', 'installmentrate',
       'statussex', 'otherdebtors', 'residencesince', 'property', 'age',
       'otherinstallmentplans', 'housing', 'existingcredits', 'job',
       'peopleliable', 'telephone', 'foreignworker', 'classification'],
      dtype='object')
